https://github.com/nipy/nipype/blob/master/examples/fmri_ants_openfmri.py

# Step 9: Connecting to Databases

In [ ]:
from os.path import abspath as opap

from nipype.interfaces.io import XNATSource
from nipype.pipeline.engine import Node, Workflow
from nipype.interfaces.fsl import BET

subject_id = 'xnat_S00001'

dg = Node(XNATSource(infields=['subject_id'],
                     outfields=['struct'],
                     config='/Users/satra/xnat_configs/nitrc_ir_config'),
          name='xnatsource')
dg.inputs.query_template = ('/projects/fcon_1000/subjects/%s/experiments/xnat_E00001'
                            '/scans/%s/resources/NIfTI/files')
dg.inputs.query_template_args['struct'] = [['subject_id', 'anat_mprage_anonymized']]
dg.inputs.subject_id = subject_id

bet = Node(BET(), name='skull_stripper')

wf = Workflow(name='testxnat')
wf.base_dir = opap('xnattest')
wf.connect(dg, 'struct', bet, 'in_file')

In [ ]:
from nipype.interfaces.io import XNATSink

ds = Node(XNATSink(config='/Users/satra/xnat_configs/central_config'),
          name='xnatsink')
ds.inputs.project_id = 'NPTEST'
ds.inputs.subject_id = 'NPTEST_xnat_S00001'
ds.inputs.experiment_id = 'test_xnat'
ds.inputs.reconstruction_id = 'bet'
ds.inputs.share = True
wf.connect(bet, 'out_file', ds, 'brain')